In [2]:
import tensorflow as tf
import os
import zipfile
from os import path, getcwd, chdir

path = f"{getcwd()}/happy-or-sad.zip"

zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

DESIRED_ACCURACY = 0.999

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>DESIRED_ACCURACY):
            print("Reaching desired accuracy so cancelling training!\n")
            self.model.stop_training=True

def train_happy_sad_model():
    callbacks = myCallback()
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        
        # The second convolution
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        
        # The third convolution
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        
        # Flatten the results to feed into a DNN
        tf.keras.layers.Flatten(),
        
        # 512 neuron hidden layer
        tf.keras.layers.Dense(512, activation='relu'),
        
        # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy',
              optimizer='adam',#RMSprop(lr=0.001),
              metrics=['acc'])

    train_datagen = ImageDataGenerator(rescale=1/255)

    train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s',
        target_size=(150, 150),
        batch_size=128,
        class_mode='binary')
        
    history = model.fit_generator(
        train_generator,
        steps_per_epoch=8,  
        epochs=15,
        verbose=1,
        callbacks = [callbacks])
        
    return history.history['acc'][-1]

In [4]:
# The Expected output: "Reached 99.9% accuracy so cancelling training!""
train_happy_sad_model()

Found 80 images belonging to 2 classes.
Epoch 1/15
8/8 [==============================] - 28s 3s/step - loss: 0.8938 - acc: 0.4875
Epoch 2/15
8/8 [==============================] - 23s 3s/step - loss: 0.5752 - acc: 0.8234
Epoch 3/15
8/8 [==============================] - 25s 3s/step - loss: 0.2028 - acc: 0.9469
Epoch 4/15
8/8 [==============================] - 23s 3s/step - loss: 0.0617 - acc: 0.9812
Epoch 5/15
7/8 [=========================>....] - ETA: 2s - loss: 0.0168 - acc: 1.0000Reaching desired accuracy so cancelling training!

8/8 [==============================] - 16s 2s/step - loss: 0.0156 - acc: 1.0000


1.0